# Feature Analysis for Steel Casting Defect Prediction

**Date:** 2025-01-21  
**Author:** Steel Defect Demo Team  
**Purpose:** Comprehensive analysis of features for defect prediction

## Overview
This notebook provides a comprehensive analysis of the steel casting dataset, focusing on:
- Feature distributions comparing normal vs defect casts
- Correlation analysis with visualization
- Feature importance ranking using multiple methods
- Univariate and bivariate feature analysis
- Feature selection recommendations

In [ ]:
# Feature Analysis for Steel Casting Defect Prediction
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.feature_selection import mutual_info_classif, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', 20)
pd.set_option('display.float_format', '{:.4f}'.format)

print("Feature Analysis Notebook - Steel Casting Defect Prediction")
print("="*60)

In [ ]:
# Load the sample data
df = pd.read_csv('../data/examples/steel_defect_sample.csv')

print(f"Dataset shape: {df.shape}")
print(f"Target distribution:\n{df['defect'].value_counts()}")
print(f"Target proportions:\n{df['defect'].value_counts(normalize=True)}")

# Basic dataset information
print("\nDataset columns:")
print(df.columns.tolist())

# Check for missing values
missing_data = df.isnull().sum()
if missing_data.sum() > 0:
    print(f"\nMissing values:\n{missing_data[missing_data > 0]}")
else:
    print("\nNo missing values found")

# Separate features and target
X = df.drop('defect', axis=1)
y = df['defect']

print(f"\nFeatures ({len(X.columns)}): {X.columns.tolist()}")
print(f"Target variable: defect (0: Normal, 1: Defect)")

In [ ]:
# === FEATURE DISTRIBUTION ANALYSIS ===
print("1. FEATURE DISTRIBUTION ANALYSIS")
print("="*50)

def analyze_feature_distributions(df, target_col='defect'):
    """
    Analyze feature distributions comparing normal vs defect casts
    """
    results = {}
    
    # Separate normal and defect samples
    normal_data = df[df[target_col] == 0]
    defect_data = df[df[target_col] == 1]
    
    feature_cols = df.columns.drop(target_col).tolist()
    
    for feature in feature_cols:
        # Statistical tests
        try:
            stat_test = stats.mannwhitneyu(
                normal_data[feature].dropna(),
                defect_data[feature].dropna(),
                alternative='two-sided'
            )
        except:
            stat_test = type('obj', (object,), {'statistic': 0, 'pvalue': 1})
        
        # Effect size (Cohen's d)
        normal_mean = normal_data[feature].mean()
        defect_mean = defect_data[feature].mean()
        normal_std = normal_data[feature].std()
        defect_std = defect_data[feature].std()
        
        pooled_std = np.sqrt(
            ((len(normal_data[feature]) - 1) * normal_std**2 +
             (len(defect_data[feature]) - 1) * defect_std**2) /
            (len(normal_data[feature]) + len(defect_data[feature]) - 2)
        )
        cohens_d = (defect_mean - normal_mean) / pooled_std if pooled_std > 0 else 0
        
        results[feature] = {
            'normal_mean': normal_mean,
            'defect_mean': defect_mean,
            'normal_std': normal_std,
            'defect_std': defect_std,
            'mannwhitney_statistic': stat_test.statistic,
            'mannwhitney_pvalue': stat_test.pvalue,
            'cohens_d': cohens_d,
            'abs_cohens_d': abs(cohens_d)
        }
    
    return pd.DataFrame(results).T

# Perform distribution analysis
distribution_results = analyze_feature_distributions(df)
distribution_results = distribution_results.sort_values('abs_cohens_d', ascending=False)

print("Feature distribution analysis results:")
print(distribution_results.round(4))

# Visualize top discriminative features
top_features = distribution_results.head(8).index.tolist()

fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.ravel()

for i, feature in enumerate(top_features):
    ax = axes[i]
    
    # Plot distributions
    normal_data = df[df['defect'] == 0][feature]
    defect_data = df[df['defect'] == 1][feature]
    
    ax.hist(normal_data, alpha=0.7, label='Normal', bins=30, density=True, color='blue')
    ax.hist(defect_data, alpha=0.7, label='Defect', bins=30, density=True, color='red')
    
    cohens_d = distribution_results.loc[feature, 'cohens_d']
    ax.set_title(f'{feature}\nCohen\'s d = {cohens_d:.3f}')
    ax.set_xlabel(feature)
    ax.set_ylabel('Density')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Feature Distributions: Normal vs Defect Casts', fontsize=16, y=1.02)
plt.show()

In [ ]:
# === CORRELATION ANALYSIS ===
print("2. CORRELATION ANALYSIS")
print("="*30)

# Calculate correlation matrix
correlation_matrix = df.corr()
target_correlations = correlation_matrix['defect'].drop('defect').abs().sort_values(ascending=False)

print("Features most correlated with defect:")
print(target_correlations.round(4))

# Correlation heatmap
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(
    correlation_matrix,
    mask=mask,
    annot=True,
    cmap='RdBu_r',
    center=0,
    square=True,
    fmt='.2f',
    cbar_kws={"shrink": .8}
)
plt.title('Feature Correlation Matrix', fontsize=14)
plt.tight_layout()
plt.show()

# High correlation pairs (potential multicollinearity)
def find_high_correlations(corr_matrix, threshold=0.8):
    """Find pairs of features with high correlation"""
    high_corr_pairs = []
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_val = corr_matrix.iloc[i, j]
            if abs(corr_val) > threshold:
                high_corr_pairs.append({
                    'feature1': corr_matrix.columns[i],
                    'feature2': corr_matrix.columns[j],
                    'correlation': corr_val
                })
    
    return pd.DataFrame(high_corr_pairs)

high_corr_pairs = find_high_correlations(correlation_matrix, threshold=0.7)
if not high_corr_pairs.empty:
    print(f"\nHigh correlation pairs (|r| > 0.7): {len(high_corr_pairs)}")
    print(high_corr_pairs)
else:
    print("\nNo high correlation pairs found (|r| > 0.7)")

In [ ]:
# === FEATURE IMPORTANCE ANALYSIS ===
print("3. FEATURE IMPORTANCE ANALYSIS")
print("="*35)

def calculate_feature_importance(X, y, random_state=42):
    """
    Calculate feature importance using multiple methods
    """
    importance_results = pd.DataFrame(index=X.columns)
    
    # 1. Random Forest Importance
    rf = RandomForestClassifier(n_estimators=100, random_state=random_state)
    rf.fit(X, y)
    importance_results['random_forest'] = rf.feature_importances_
    
    # 2. Mutual Information
    mi_scores = mutual_info_classif(X, y, random_state=random_state)
    importance_results['mutual_information'] = mi_scores
    
    # 3. F-statistics
    f_scores, f_pvalues = f_classif(X, y)
    importance_results['f_statistic'] = f_scores
    importance_results['f_pvalue'] = f_pvalues
    
    # 4. Correlation with target
    importance_results['correlation'] = abs(X.corrwith(y))
    
    # Normalize importance scores to 0-1 range for comparison
    for col in ['random_forest', 'mutual_information', 'f_statistic', 'correlation']:
        if col in importance_results.columns:
            max_val = importance_results[col].max()
            if max_val > 0:
                importance_results[f'{col}_normalized'] = importance_results[col] / max_val
    
    # Calculate composite importance score
    importance_cols = [col for col in importance_results.columns if col.endswith('_normalized')]
    importance_results['composite_importance'] = importance_results[importance_cols].mean(axis=1)
    
    return importance_results

# Calculate feature importance
importance_results = calculate_feature_importance(X, y)
importance_results = importance_results.sort_values('composite_importance', ascending=False)

print("Feature importance results:")
print(importance_results[['random_forest', 'mutual_information', 'correlation', 'composite_importance']].round(4))

# Visualize feature importance
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Random Forest Importance
axes[0, 0].barh(range(len(X.columns)), importance_results['random_forest'])
axes[0, 0].set_yticks(range(len(X.columns)))
axes[0, 0].set_yticklabels(importance_results.index)
axes[0, 0].set_title('Random Forest Feature Importance')
axes[0, 0].set_xlabel('Importance')

# Mutual Information
axes[0, 1].barh(range(len(X.columns)), importance_results['mutual_information'])
axes[0, 1].set_yticks(range(len(X.columns)))
axes[0, 1].set_yticklabels(importance_results.index)
axes[0, 1].set_title('Mutual Information Scores')
axes[0, 1].set_xlabel('MI Score')

# Correlation
axes[1, 0].barh(range(len(X.columns)), importance_results['correlation'])
axes[1, 0].set_yticks(range(len(X.columns)))
axes[1, 0].set_yticklabels(importance_results.index)
axes[1, 0].set_title('Correlation with Target')
axes[1, 0].set_xlabel('Absolute Correlation')

# Composite Importance
axes[1, 1].barh(range(len(X.columns)), importance_results['composite_importance'])
axes[1, 1].set_yticks(range(len(X.columns)))
axes[1, 1].set_yticklabels(importance_results.index)
axes[1, 1].set_title('Composite Importance Score')
axes[1, 1].set_xlabel('Composite Score')

plt.tight_layout()
plt.suptitle('Feature Importance Comparison', fontsize=16, y=1.02)
plt.show()

In [ ]:
# === FEATURE SELECTION RECOMMENDATIONS ===
print("4. FEATURE SELECTION RECOMMENDATIONS")
print("="*40)

# Performance comparison of different feature sets
def evaluate_feature_sets(X, y, feature_sets, cv=5):
    """Evaluate different feature sets using cross-validation"""
    results = {}
    
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    
    for set_name, features in feature_sets.items():
        if len(features) > 0 and all(f in X.columns for f in features):
            X_subset = X[features]
            scores = cross_val_score(rf, X_subset, y, cv=cv, scoring='roc_auc')
            results[set_name] = {
                'n_features': len(features),
                'mean_auc': scores.mean(),
                'std_auc': scores.std(),
                'features': features
            }
    
    return results

# Define feature sets to evaluate
feature_sets = {
    'all_features': X.columns.tolist(),
    'top_5_importance': importance_results.head(5).index.tolist(),
    'top_8_importance': importance_results.head(8).index.tolist(),
    'top_3_correlation': target_correlations.head(3).index.tolist(),
    'top_5_correlation': target_correlations.head(5).index.tolist()
}

# Evaluate feature sets
evaluation_results = evaluate_feature_sets(X, y, feature_sets)

print("Feature Set Performance Comparison:")
performance_df = pd.DataFrame({
    name: {
        'n_features': results['n_features'],
        'mean_auc': results['mean_auc'],
        'std_auc': results['std_auc']
    }
    for name, results in evaluation_results.items()
}).T

performance_df = performance_df.sort_values('mean_auc', ascending=False)
print(performance_df.round(4))

# Visualize performance comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# AUC comparison
ax1.bar(range(len(performance_df)), performance_df['mean_auc'], 
        yerr=performance_df['std_auc'], capsize=5)
ax1.set_xticks(range(len(performance_df)))
ax1.set_xticklabels(performance_df.index, rotation=45)
ax1.set_ylabel('Cross-Validation AUC')
ax1.set_title('Feature Set Performance Comparison')
ax1.grid(True, alpha=0.3)

# Number of features vs AUC
ax2.scatter(performance_df['n_features'], performance_df['mean_auc'], s=100)
for idx, row in performance_df.iterrows():
    ax2.annotate(idx, (row['n_features'], row['mean_auc']), 
                xytext=(5, 5), textcoords='offset points')
ax2.set_xlabel('Number of Features')
ax2.set_ylabel('Cross-Validation AUC')
ax2.set_title('Features vs Performance Trade-off')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Recommendations
print("\n" + "="*50)
print("FEATURE SELECTION RECOMMENDATIONS")
print("="*50)

best_set = performance_df.index[0]
print(f"\n🏆 Best performing feature set: {best_set}")
print(f"   • Number of features: {evaluation_results[best_set]['n_features']}")
print(f"   • Cross-validation AUC: {evaluation_results[best_set]['mean_auc']:.4f} ± {evaluation_results[best_set]['std_auc']:.4f}")
print(f"   • Features: {evaluation_results[best_set]['features']}")

print(f"\n📊 Top features by different methods:")
print(f"   • By composite importance: {importance_results.head(5).index.tolist()}")
print(f"   • By correlation with target: {target_correlations.head(5).index.tolist()}")
print(f"   • By effect size (Cohen's d): {distribution_results.head(5).index.tolist()}")

print(f"\n🎯 Final recommendations:")
print(f"   1. Use top 5-8 features based on composite importance for best balance")
print(f"   2. Monitor high-correlation features for multicollinearity")
print(f"   3. Consider domain expertise for final feature selection")
print(f"   4. Validate on holdout test set before deployment")

In [ ]:
# === SUMMARY AND EXPORT ===
print("5. SUMMARY AND EXPORT")
print("="*25)

# Summary statistics
print("ANALYSIS SUMMARY")
print("="*20)
print(f"📊 Dataset: {df.shape[0]} samples, {len(X.columns)} features")
print(f"📈 Class distribution: {(y==0).sum()} normal ({(y==0).sum()/len(y)*100:.1f}%), {(y==1).sum()} defect ({(y==1).sum()/len(y)*100:.1f}%)")
print(f"🔍 Best feature by importance: {importance_results.index[0]} (score: {importance_results.iloc[0]['composite_importance']:.3f})")
print(f"📊 Best feature by correlation: {target_correlations.index[0]} (r = {target_correlations.iloc[0]:.3f})")
print(f"⚡ Best feature by effect size: {distribution_results.index[0]} (Cohen's d = {distribution_results.iloc[0]['cohens_d']:.3f})")

# Export results
print(f"\n💾 Exporting analysis results...")

# Create results directory
import os
results_dir = '../results/feature_analysis'
os.makedirs(results_dir, exist_ok=True)

# Export feature importance results
importance_results.to_csv(f'{results_dir}/feature_importance.csv')
print(f"   ✅ Feature importance saved to {results_dir}/feature_importance.csv")

# Export distribution analysis
distribution_results.to_csv(f'{results_dir}/distribution_analysis.csv')
print(f"   ✅ Distribution analysis saved to {results_dir}/distribution_analysis.csv")

# Export correlation matrix
correlation_matrix.to_csv(f'{results_dir}/correlation_matrix.csv')
print(f"   ✅ Correlation matrix saved to {results_dir}/correlation_matrix.csv")

# Export performance comparison
performance_df.to_csv(f'{results_dir}/feature_set_performance.csv')
print(f"   ✅ Feature set performance saved to {results_dir}/feature_set_performance.csv")

# Export recommended features
recommended_features = {
    'top_5_importance': importance_results.head(5).index.tolist(),
    'top_8_importance': importance_results.head(8).index.tolist(),
    'top_5_correlation': target_correlations.head(5).index.tolist(),
    'best_performing_set': evaluation_results[best_set]['features']
}

import json
with open(f'{results_dir}/recommended_features.json', 'w') as f:
    json.dump(recommended_features, f, indent=2)
print(f"   ✅ Recommended features saved to {results_dir}/recommended_features.json")

print(f"\n🎉 Feature analysis complete! Results exported to {results_dir}/")
print(f"\n📋 Key takeaways:")
print(f"   • Use {len(evaluation_results[best_set]['features'])} features for optimal performance")
print(f"   • Expected AUC: {evaluation_results[best_set]['mean_auc']:.3f}")
print(f"   • Focus on: {', '.join(importance_results.head(3).index.tolist())}")